In [1]:
batch_size_for_classification = 3000
#prediction
prediction_path = "Export/cdnow/cdnow_prediction_trmf_variantB_"+str(batch_size_for_classification+2000)+'_.csv'

In [2]:
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from sklearn.base import clone
from sklearn.model_selection import ParameterGrid
import networkx as nx
from sklearn.preprocessing import MinMaxScaler
from trmf import TRMFRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import datetime
import random
import rpy2
import math
import trmf
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
import time
import pickle
import warnings
warnings.filterwarnings("ignore")
from rpy2.robjects.packages import importr
import rpy2.robjects.packages as rpackages
import rpy2.robjects as robjects
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1) # select the first mirror in the list
packnames = ['forecast']
from rpy2.robjects.vectors import StrVector
try:
    base = importr('forecast')
except:
    utils.install_packages(StrVector('forecast'))
    base = importr('forecast')

In [3]:
# Path fo import files
#input data
import_data = "import/CDNOW_master.txt"

#prediction
prediction_path = "Export/cdnow/cdnow_prediction_variantB.csv"
# clusters
import_clusters = "clusters/results_cdnow.csv"

### 0. Preprocessing alibaba + normalizing

In [4]:
df = pd.read_csv(import_data)

In [5]:
df = pd.read_csv(import_data,header=None,names  =['id','Date','disks','price_purchase'],delim_whitespace=True,)
df['Date'] = df["Date"].apply(lambda x:datetime.datetime.strptime(str(x),"%Y%m%d"))
df = df.drop(columns = ['disks'])
df = df.groupby(by=['id','Date']).sum().reset_index()
df = df.pivot(columns = 'Date',index = 'id',values = 'price_purchase')
df.sample(2)

Date,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,1997-01-06 00:00:00,1997-01-07 00:00:00,1997-01-08 00:00:00,1997-01-09 00:00:00,1997-01-10 00:00:00,...,1998-06-21 00:00:00,1998-06-22 00:00:00,1998-06-23 00:00:00,1998-06-24 00:00:00,1998-06-25 00:00:00,1998-06-26 00:00:00,1998-06-27 00:00:00,1998-06-28 00:00:00,1998-06-29 00:00:00,1998-06-30 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
680,NaN,NaN,15.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 0.1 Choose TS with given clusters and divide into test/train dataset

In [6]:
cluster_TS  = pd.read_csv(import_clusters)
cluster_TS.head()

,Unnamed: 0,clusters_r,clusters_m,clusters_f
0,11770,0,0,0
1,14675,17,0,8
2,10026,2,0,3
3,13997,1,0,2
4,741,0,0,0


In [7]:
df_train = cluster_TS.merge(df,left_on = "Unnamed: 0", right_on = 'id')

In [8]:
df_train = df_train.rename(index = str, columns={"Unnamed: 0": "id"}).set_index('id')
df_train.head()

,clusters_r,clusters_m,clusters_f,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,1997-01-06 00:00:00,1997-01-07 00:00:00,...,1998-06-21 00:00:00,1998-06-22 00:00:00,1998-06-23 00:00:00,1998-06-24 00:00:00,1998-06-25 00:00:00,1998-06-26 00:00:00,1998-06-27 00:00:00,1998-06-28 00:00:00,1998-06-29 00:00:00,1998-06-30 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
11770,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14675,17,0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10026,2,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13997,1,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
741,0,0,0,NaN,NaN,NaN,43.13,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# for this we will predict clusters
df_test= df.drop(df_train.index.values,axis=0).sample(batch_size_for_classification)
df_test.head()

Date,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,1997-01-06 00:00:00,1997-01-07 00:00:00,1997-01-08 00:00:00,1997-01-09 00:00:00,1997-01-10 00:00:00,...,1998-06-21 00:00:00,1998-06-22 00:00:00,1998-06-23 00:00:00,1998-06-24 00:00:00,1998-06-25 00:00:00,1998-06-26 00:00:00,1998-06-27 00:00:00,1998-06-28 00:00:00,1998-06-29 00:00:00,1998-06-30 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
10063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### NAN -> 0

In [10]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)
df_train.sample(2)

,clusters_r,clusters_m,clusters_f,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,1997-01-06 00:00:00,1997-01-07 00:00:00,...,1998-06-21 00:00:00,1998-06-22 00:00:00,1998-06-23 00:00:00,1998-06-24 00:00:00,1998-06-25 00:00:00,1998-06-26 00:00:00,1998-06-27 00:00:00,1998-06-28 00:00:00,1998-06-29 00:00:00,1998-06-30 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
15492,6,0,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7468,0,0,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Output File:
result = pd.DataFrame()

# 1. Prediction with given clusters on train data (2000 rows)

In [12]:
train_targets, test_targets= train_test_split(df_train.transpose(), test_size=.127, shuffle=False)
train_targets = train_targets.transpose()
test_targets = test_targets.transpose()

### 1.0.1 GridSearch function for TMRF

In [13]:
base = TRMFRegressor(n_components=1,
                 n_order=0,
                 fit_regression=False,
                 fit_intercept=True,
                 nonnegative_factors=True,
                 n_max_mf_iter=5)


def gridsearch_trmf(X,test_targets):
    grid = ParameterGrid(dict(
    n_components=np.r_[1:2],
    n_order=np.r_[1:2],
    C_Z=np.logspace(-2, 1,num=1),
    C_F=np.logspace(-2, 1,num=1),
    C_phi=np.logspace(-2, 1, num=1),
    eta_Z=np.linspace(0.05, 0.95,num=1),
    ))
    
    transformer = StandardScaler(with_mean=True, with_std=True)
    X = transformer.fit_transform(X)

    
    # run the experiment in parallel
    par_ = Parallel(n_jobs=-1, verbose=0)
    results = par_(delayed(helper)(par, X, test_targets.transpose(),transformer) for par in grid)
    
    keys = ['n_order', 'n_components', 'eta_Z', 'C_phi', 'C_Z', 'C_F']
    data = dict((tuple(par[k] for k in keys), rmse,) for par, rmse in results)
    sr = pd.Series(data, name="rmse").sort_index().rename_axis(keys)
    cube = sr.values.reshape(*[len(grid.param_grid[0][k]) for k in keys])
    stepping = [grid.param_grid[0][k] for k in keys]
    # find the flat index of the smallest value
    flat_index = np.argmin(cube)
    # ... and unravel into into a multidimensional index
    index = np.unravel_index(flat_index, cube.shape)
    # collect the best paramaters from the grid
    best_ = {k: grid.param_grid[0][k][i] for k, i in zip(keys, index)}
    return best_

transformer = StandardScaler(with_mean=True, with_std=True)


def helper(par, train, test,  transformer,base=base):
    basee = TRMFRegressor(n_components=1,
                     n_order=0,
                     fit_regression=False,
                     fit_intercept=True,
                     nonnegative_factors=True,
                     n_max_mf_iter=5)
    
    # clone, set parameters and fit
  #  print(train.shape)
    #print(test.shape)
    trmf = clone(basee).set_params(**par).fit(train)
    
    # predict and return
    pred = transformer.inverse_transform(
        trmf.predict(n_ahead=len(test)))

    return par, mean_squared_error(test, pred)

## 1.1 TMRF

In [14]:
n_ahead = test_targets.shape[1]

In [15]:
train_targets.head()

,clusters_r,clusters_m,clusters_f,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,1997-01-06 00:00:00,1997-01-07 00:00:00,...,1998-04-12 00:00:00,1998-04-13 00:00:00,1998-04-14 00:00:00,1998-04-15 00:00:00,1998-04-16 00:00:00,1998-04-17 00:00:00,1998-04-18 00:00:00,1998-04-19 00:00:00,1998-04-20 00:00:00,1998-04-21 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
11770,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
14675,17.0,0.0,8.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10026,2.0,0.0,3.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13997,1.0,0.0,2.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
741,0.0,0.0,0.0,0.0,0.0,0.0,43.13,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 1.1.1 Recency

In [16]:
%%time
# Create TRMF Models
for cluster_number in df_train.clusters_r.unique():
    X = train_targets.loc[train_targets.clusters_r == cluster_number].drop(columns = ['clusters_m','clusters_f'])
    true_values = test_targets.loc[ X.index]
    
    
    # Scaling (Not minmax because of it wasn't used in trmf github)
    transformer = StandardScaler(with_mean=True, with_std=True)
    X = X.transpose()
    X = transformer.fit_transform(X)
    #GridSearchCV for TRMF
    best = gridsearch_trmf(X,true_values)
    #X = X.transpose()
    
    cluster = TRMFRegressor(**best, eta_F=0., adj=None, C_B=0., fit_regression=False,
                     fit_intercept=True, nonnegative_factors=True, n_max_mf_iter=5,
                     z_step_kind="tron")
    
    
    
    cluster.fit(X)
    #predicted = cluster.predict(X=X, n_ahead=n_ahead)
    predicted = transformer.inverse_transform( cluster.predict(X=X, n_ahead=n_ahead))
    X = X.transpose()
    df_prediction = pd.DataFrame(predicted).transpose()
    df_prediction.columns  = true_values.columns 
    df_prediction.index = true_values.index
    
    # Transform prediction for output
    #MODEL
    df_prediction.insert(0,'RMSE',math.sqrt(mean_squared_error(true_values,df_prediction)))
    
    df_prediction.insert(0,'model','trmf')
    #METHOD
    df_prediction.insert(0,'method','clustered')
    #STEP
    df_prediction.insert(0,'step','before_classifier')
    # CLUSTER
    df_prediction.insert(0,'cluster_id','cluster_'+str(cluster_number)+'_R')
    #Wide to logn
    df_prediction = df_prediction.reset_index().melt(id_vars =['id','cluster_id','step','method','model','RMSE']).rename(index = str, columns={'variable':'date','value':'predicted_value'})
    true_values=true_values.reset_index().melt(id_vars=['id']).rename(index = str, columns={'variable':'date','value':'true_value'})
    
    #break
    result = result.append(df_prediction.merge(true_values,left_on = ['id','date'],right_on=['id','date']))

CPU times: user 1min 15s, sys: 8min 39s, total: 9min 55s
Wall time: 2min 2s


### 1.1.2 Monetary

In [17]:
%%time
# Create TRMF Models
for cluster_number in df_train.clusters_m.unique():
    X = train_targets.loc[train_targets.clusters_m == cluster_number].drop(columns = ['clusters_r','clusters_f'])
    true_values = test_targets.loc[ X.index]
    
    
    # Scaling (Not minmax because of it wasn't used in trmf github)
    transformer = StandardScaler(with_mean=True, with_std=True)
    X = X.transpose()
    X = transformer.fit_transform(X)
    #GridSearchCV for TRMF
    best = gridsearch_trmf(X,true_values)
    #X = X.transpose()
    
    cluster = TRMFRegressor(**best, eta_F=0., adj=None, C_B=0., fit_regression=False,
                     fit_intercept=True, nonnegative_factors=True, n_max_mf_iter=5,
                     z_step_kind="tron")
    
    
    
    cluster.fit(X)
    #predicted = cluster.predict(X=X, n_ahead=n_ahead)
    predicted = transformer.inverse_transform( cluster.predict(X=X, n_ahead=n_ahead))
    X = X.transpose()
    df_prediction = pd.DataFrame(predicted).transpose()
    df_prediction.columns  = true_values.columns 
    df_prediction.index = true_values.index
    
    # Transform prediction for output
    #MODEL
    df_prediction.insert(0,'RMSE',math.sqrt(mean_squared_error(true_values,df_prediction)))
    
    df_prediction.insert(0,'model','trmf')
    #METHOD
    df_prediction.insert(0,'method','clustered')
    #STEP
    df_prediction.insert(0,'step','before_classifier')
    # CLUSTER
    df_prediction.insert(0,'cluster_id','cluster_'+str(cluster_number)+'_M')
    #Wide to logn
    df_prediction = df_prediction.reset_index().melt(id_vars =['id','cluster_id','step','method','model','RMSE']).rename(index = str, columns={'variable':'date','value':'predicted_value'})
    true_values=true_values.reset_index().melt(id_vars=['id']).rename(index = str, columns={'variable':'date','value':'true_value'})
    
    #break
    result = result.append(df_prediction.merge(true_values,left_on = ['id','date'],right_on=['id','date']))

CPU times: user 13.9 s, sys: 1min 28s, total: 1min 42s
Wall time: 16.5 s


### 1.1.3 Frequency

In [18]:
%%time
# Create TRMF Models
for cluster_number in df_train.clusters_f.unique():
    X = train_targets.loc[train_targets.clusters_f == cluster_number].drop(columns = ['clusters_r','clusters_f'])
    true_values = test_targets.loc[ X.index]
    
    
    # Scaling (Not minmax because of it wasn't used in trmf github)
    transformer = StandardScaler(with_mean=True, with_std=True)
    X = X.transpose()
    X = transformer.fit_transform(X)
    #GridSearchCV for TRMF
    best = gridsearch_trmf(X,true_values)
    #X = X.transpose()
    
    cluster = TRMFRegressor(**best, eta_F=0., adj=None, C_B=0., fit_regression=False,
                     fit_intercept=True, nonnegative_factors=True, n_max_mf_iter=5,
                     z_step_kind="tron")
    
    
    
    cluster.fit(X)
    #predicted = cluster.predict(X=X, n_ahead=n_ahead)
    predicted = transformer.inverse_transform( cluster.predict(X=X, n_ahead=n_ahead))
    X = X.transpose()
    df_prediction = pd.DataFrame(predicted).transpose()
    df_prediction.columns  = true_values.columns 
    df_prediction.index = true_values.index
    
    # Transform prediction for output
    #MODEL
    df_prediction.insert(0,'RMSE',math.sqrt(mean_squared_error(true_values,df_prediction)))
    
    df_prediction.insert(0,'model','trmf')
    #METHOD
    df_prediction.insert(0,'method','clustered')
    #STEP
    df_prediction.insert(0,'step','before_classifier')
    # CLUSTER
    df_prediction.insert(0,'cluster_id','cluster_'+str(cluster_number)+'_F')
    #Wide to logn
    df_prediction = df_prediction.reset_index().melt(id_vars =['id','cluster_id','step','method','model','RMSE']).rename(index = str, columns={'variable':'date','value':'predicted_value'})
    true_values=true_values.reset_index().melt(id_vars=['id']).rename(index = str, columns={'variable':'date','value':'true_value'})
    
    #break
    result = result.append(df_prediction.merge(true_values,left_on = ['id','date'],right_on=['id','date']))

CPU times: user 1min 33s, sys: 10min 52s, total: 12min 25s
Wall time: 2min 9s


## 1.2 Theta

### 1.2.1 Recency

In [19]:
%%time
base = importr('forecast')
# Create Theta Models
for cluster_number in df_train.clusters_r.unique():
    X = train_targets.loc[train_targets.clusters_r == cluster_number].drop(columns = ['clusters_m','clusters_f'])
    true_values = test_targets.loc[ X.index]
    # Scaling
    scl =MinMaxScaler() #StandardScaler(with_mean=True, with_std=True)
    X = scl.fit_transform(X)
    #THETA
    df_prediction = pd.DataFrame(columns = test_targets.columns)

    train_targets_scaled = pd.DataFrame(X)
    train_targets_scaled.index = true_values.index


    for index, row in train_targets_scaled.iterrows():
        res = robjects.FloatVector(row.drop(columns = ['uid']))
        r_prediction = base.thetaf(res,70)[1]
        df_prediction.loc[index] = r_prediction
    #Transform prediction for output 
    #MODEL
    df_prediction.insert(0,'RMSE',math.sqrt(mean_squared_error(true_values,df_prediction)))
    
    df_prediction.insert(0,'model','theta')
    #METHOD
    df_prediction.insert(0,'method','clustered')
    #STEP
    df_prediction.insert(0,'step','before_classifier')
    # CLUSTER
    df_prediction.insert(0,'cluster_id','cluster_'+str(cluster_number)+'_R')
    #Wide to logn
    df_prediction = df_prediction.reset_index().melt(id_vars =['index','cluster_id','step','method','model','RMSE']).rename(index = str, columns={'variable':'date','value':'predicted_value','index':'id'})
    true_values=true_values.reset_index().melt(id_vars=['id']).rename(index = str, columns={'variable':'date','value':'true_value'})
    
    #break
    result = result.append(df_prediction.merge(true_values,left_on = ['id','date'],right_on=['id','date']))

CPU times: user 21.1 s, sys: 100 ms, total: 21.2 s
Wall time: 21.2 s


### 1.2.2 Monetary

In [20]:
%%time
# Create Theta Models
for cluster_number in df_train.clusters_m.unique():
    X = train_targets.loc[train_targets.clusters_m == cluster_number].drop(columns = ['clusters_r','clusters_f'])
    true_values = test_targets.loc[ X.index]
    # Scaling
    scl =MinMaxScaler() #StandardScaler(with_mean=True, with_std=True)
    X = scl.fit_transform(X)
    #THETA
    df_prediction = pd.DataFrame(columns = test_targets.columns)

    train_targets_scaled = pd.DataFrame(X)
    train_targets_scaled.index = true_values.index


    for index, row in train_targets_scaled.iterrows():
        res = robjects.FloatVector(row.drop(columns = ['uid']))
        r_prediction = base.thetaf(res,70)[1]
        df_prediction.loc[index] = r_prediction
    #Transform prediction for output 
    #MODEL
    df_prediction.insert(0,'RMSE',math.sqrt(mean_squared_error(true_values,df_prediction)))
    
    df_prediction.insert(0,'model','theta')
    #METHOD
    df_prediction.insert(0,'method','clustered')
    #STEP
    df_prediction.insert(0,'step','before_classifier')
    # CLUSTER
    df_prediction.insert(0,'cluster_id','cluster_'+str(cluster_number)+'_M')
    #Wide to logn
    df_prediction = df_prediction.reset_index().melt(id_vars =['index','cluster_id','step','method','model','RMSE']).rename(index = str, columns={'variable':'date','value':'predicted_value','index':'id'})
    true_values=true_values.reset_index().melt(id_vars=['id']).rename(index = str, columns={'variable':'date','value':'true_value'})
    
    #break
    result = result.append(df_prediction.merge(true_values,left_on = ['id','date'],right_on=['id','date']))

CPU times: user 19.6 s, sys: 0 ns, total: 19.6 s
Wall time: 19.6 s


### 1.2.3 Frequency

In [21]:
%%time
# Create Theta Models
for cluster_number in df_train.clusters_f.unique():
    X = train_targets.loc[train_targets.clusters_f == cluster_number].drop(columns = ['clusters_r','clusters_m'])
    true_values = test_targets.loc[ X.index]
    # Scaling
    scl =MinMaxScaler() #StandardScaler(with_mean=True, with_std=True)
    X = scl.fit_transform(X)
    #THETA
    df_prediction = pd.DataFrame(columns = test_targets.columns)

    train_targets_scaled = pd.DataFrame(X)
    train_targets_scaled.index = true_values.index


    for index, row in train_targets_scaled.iterrows():
        res = robjects.FloatVector(row.drop(columns = ['uid']))
        r_prediction = base.thetaf(res,70)[1]
        df_prediction.loc[index] = r_prediction
    #Transform prediction for output 
    #MODEL
    df_prediction.insert(0,'RMSE',math.sqrt(mean_squared_error(true_values,df_prediction)))
    
    df_prediction.insert(0,'model','theta')
    #METHOD
    df_prediction.insert(0,'method','clustered')
    #STEP
    df_prediction.insert(0,'step','before_classifier')
    # CLUSTER
    df_prediction.insert(0,'cluster_id','cluster_'+str(cluster_number)+'_F')
    #Wide to logn
    df_prediction = df_prediction.reset_index().melt(id_vars =['index','cluster_id','step','method','model','RMSE']).rename(index = str, columns={'variable':'date','value':'predicted_value','index':'id'})
    true_values=true_values.reset_index().melt(id_vars=['id']).rename(index = str, columns={'variable':'date','value':'true_value'})
    
    #break
    result = result.append(df_prediction.merge(true_values,left_on = ['id','date'],right_on=['id','date']))

CPU times: user 20.8 s, sys: 32 ms, total: 20.8 s
Wall time: 20.8 s


## 2 Training the classifier to predict cluster

In [22]:
# use the classifier on the same length as the length used for training (all minus 70)
# Data for training Classifier
X_2000_train,test_2000 = train_test_split(df_train.transpose(), test_size=.127, shuffle=False)
X_2000_train = X_2000_train.transpose()
test_2000 = test_2000.transpose()
# Predict clusters using these data
X_600_train, test_600= train_test_split(df_test.transpose(), test_size=.127, shuffle=False)
X_600_train = X_600_train.transpose()


test_600.transpose()

y_train_2000_r = df_train.clusters_r
y_train_2000_f = df_train.clusters_f
y_train_2000_m = df_train.clusters_m

X_2000_train = X_2000_train.drop(columns = ['clusters_r','clusters_m','clusters_f'])
X_600_train.columns = X_2000_train.columns

In [23]:
from xgboost import XGBClassifier 
params = {
        'min_child_weight': [1],# 5, 10],
        'gamma': [0.5],#, 1, 2, 5],
        'subsample': [0.6],# 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8],#, 1.0],
        'max_depth': [3]
        }



### 2.1 Recency

In [24]:
%%time
clf = XGBClassifier( nthread=6,  seed=0, silent=True)
clf = GridSearchCV(clf,params,cv=3,n_jobs=-1,verbose=2)
clf.fit(X_2000_train,y_train_2000_r)
xgb = clf.best_estimator_
xgb.fit(X_2000_train,y_train_2000_r)
predicted_clusters = xgb.predict(X_600_train)

df_test.insert(0,'clusters_r',predicted_clusters)
#12 fits - 2 minutes

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:   15.4s remaining:   15.4s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   16.1s finished


CPU times: user 1min 3s, sys: 928 ms, total: 1min 4s
Wall time: 27.5 s


### 2.2 Monetary

In [25]:
%%time
clf = XGBClassifier( nthread=6,  seed=0, silent=True)
clf = GridSearchCV(clf,params,cv=3,n_jobs=-1,verbose=2)
clf.fit(X_2000_train,y_train_2000_m)
xgb = clf.best_estimator_
xgb.fit(X_2000_train,y_train_2000_m)
predicted_clusters = xgb.predict(X_600_train)

df_test.insert(0,'clusters_m',predicted_clusters)
# 12 fits - 9 seconds

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    2.1s remaining:    2.1s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    2.3s finished


CPU times: user 2.21 s, sys: 0 ns, total: 2.21 s
Wall time: 2.68 s


### 2.3 Frequency

In [26]:
%%time
clf = XGBClassifier( nthread=6,  seed=0, silent=True)
clf = GridSearchCV(clf,params,cv=3,n_jobs=-1,verbose=2)
clf.fit(X_2000_train,y_train_2000_f)
xgb = clf.best_estimator_
xgb.fit(X_2000_train,y_train_2000_f)
predicted_clusters = xgb.predict(X_600_train)

df_test.insert(0,'clusters_f',predicted_clusters)
#12 fits - 90 seconds

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:   13.3s remaining:   13.3s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   14.4s finished


CPU times: user 54.9 s, sys: 0 ns, total: 54.9 s
Wall time: 23.9 s


In [27]:
df_test.sample(5)

Date,clusters_f,clusters_m,clusters_r,1997-01-01 00:00:00,1997-01-02 00:00:00,1997-01-03 00:00:00,1997-01-04 00:00:00,1997-01-05 00:00:00,1997-01-06 00:00:00,1997-01-07 00:00:00,...,1998-06-21 00:00:00,1998-06-22 00:00:00,1998-06-23 00:00:00,1998-06-24 00:00:00,1998-06-25 00:00:00,1998-06-26 00:00:00,1998-06-27 00:00:00,1998-06-28 00:00:00,1998-06-29 00:00:00,1998-06-30 00:00:00
id,,,,,,,,,,,,,,,,,,,,,
11259,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7225,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16679,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23454,9,0,15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18204,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 3.0 Prediction with predicted clusters on full data (2600 rows)

In [28]:
df_with_predicted_labels = df_train.append(df_test)

In [29]:
train_targets, test_targets= train_test_split(df_with_predicted_labels.transpose(), test_size=.127, shuffle=False)
train_targets = train_targets.transpose()
test_targets = test_targets.transpose()

## 3.1 TRMF

### 3.1.1 Recency

In [30]:
%%time
# Create TRMF Models
for cluster_number in df_with_predicted_labels.clusters_r.unique():
    X = train_targets.loc[train_targets.clusters_r == cluster_number].drop(columns = ['clusters_m','clusters_f'])
    true_values = test_targets.loc[ X.index]
    
    
    # Scaling (Not minmax because of it wasn't used in trmf github)
    transformer = StandardScaler(with_mean=True, with_std=True)
    X = X.transpose()
    X = transformer.fit_transform(X)
    #GridSearchCV for TRMF
    best = gridsearch_trmf(X,true_values)
    #X = X.transpose()
    
    cluster = TRMFRegressor(**best, eta_F=0., adj=None, C_B=0., fit_regression=False,
                     fit_intercept=True, nonnegative_factors=True, n_max_mf_iter=5,
                     z_step_kind="tron")
    
    
    
    cluster.fit(X)
    #predicted = cluster.predict(X=X, n_ahead=n_ahead)
    predicted = transformer.inverse_transform( cluster.predict(X=X, n_ahead=n_ahead))
    X = X.transpose()
    df_prediction = pd.DataFrame(predicted).transpose()
    df_prediction.columns  = true_values.columns 
    df_prediction.index = true_values.index
    
    # Transform prediction for output
    #MODEL
    df_prediction.insert(0,'RMSE',math.sqrt(mean_squared_error(true_values,df_prediction)))
    
    df_prediction.insert(0,'model','trmf')
    #METHOD
    df_prediction.insert(0,'method','clustered')
    #STEP
    df_prediction.insert(0,'step','before_classifier')
    # CLUSTER
    df_prediction.insert(0,'cluster_id','cluster_'+str(cluster_number)+'_R')
    #Wide to logn
    df_prediction = df_prediction.reset_index().melt(id_vars =['id','cluster_id','step','method','model','RMSE']).rename(index = str, columns={'variable':'date','value':'predicted_value'})
    true_values=true_values.reset_index().melt(id_vars=['id']).rename(index = str, columns={'variable':'date','value':'true_value'})
    
    #break
    result = result.append(df_prediction.merge(true_values,left_on = ['id','date'],right_on=['id','date']))

CPU times: user 1min 8s, sys: 7min 5s, total: 8min 13s
Wall time: 57 s


### 3.1.2 Monetary

In [31]:
%%time
# Create TRMF Models
for cluster_number in df_with_predicted_labels.clusters_m.unique():
    X = train_targets.loc[train_targets.clusters_m == cluster_number].drop(columns = ['clusters_r','clusters_f'])
    true_values = test_targets.loc[ X.index]
    
    
    # Scaling (Not minmax because of it wasn't used in trmf github)
    transformer = StandardScaler(with_mean=True, with_std=True)
    X = X.transpose()
    X = transformer.fit_transform(X)
    #GridSearchCV for TRMF
    best = gridsearch_trmf(X,true_values)
    #X = X.transpose()
    
    cluster = TRMFRegressor(**best, eta_F=0., adj=None, C_B=0., fit_regression=False,
                     fit_intercept=True, nonnegative_factors=True, n_max_mf_iter=5,
                     z_step_kind="tron")
    
    
    
    cluster.fit(X)
    #predicted = cluster.predict(X=X, n_ahead=n_ahead)
    predicted = transformer.inverse_transform( cluster.predict(X=X, n_ahead=n_ahead))
    X = X.transpose()
    df_prediction = pd.DataFrame(predicted).transpose()
    df_prediction.columns  = true_values.columns 
    df_prediction.index = true_values.index
    
    # Transform prediction for output
    #MODEL
    df_prediction.insert(0,'RMSE',math.sqrt(mean_squared_error(true_values,df_prediction)))
    
    df_prediction.insert(0,'model','trmf')
    #METHOD
    df_prediction.insert(0,'method','clustered')
    #STEP
    df_prediction.insert(0,'step','before_classifier')
    # CLUSTER
    df_prediction.insert(0,'cluster_id','cluster_'+str(cluster_number)+'_M')
    #Wide to logn
    df_prediction = df_prediction.reset_index().melt(id_vars =['id','cluster_id','step','method','model','RMSE']).rename(index = str, columns={'variable':'date','value':'predicted_value'})
    true_values=true_values.reset_index().melt(id_vars=['id']).rename(index = str, columns={'variable':'date','value':'true_value'})
    
    #break
    result = result.append(df_prediction.merge(true_values,left_on = ['id','date'],right_on=['id','date']))

CPU times: user 36.4 s, sys: 3min 39s, total: 4min 16s
Wall time: 24.5 s


### 3.1.3 Frequency

In [32]:
%%time
# Create TRMF Models
for cluster_number in df_with_predicted_labels.clusters_f.unique():
    X = train_targets.loc[train_targets.clusters_f == cluster_number].drop(columns = ['clusters_r','clusters_f'])
    true_values = test_targets.loc[ X.index]
    
    
    # Scaling (Not minmax because of it wasn't used in trmf github)
    transformer = StandardScaler(with_mean=True, with_std=True)
    X = X.transpose()
    X = transformer.fit_transform(X)
    #GridSearchCV for TRMF
    best = gridsearch_trmf(X,true_values)
    #X = X.transpose()
    
    cluster = TRMFRegressor(**best, eta_F=0., adj=None, C_B=0., fit_regression=False,
                     fit_intercept=True, nonnegative_factors=True, n_max_mf_iter=5,
                     z_step_kind="tron")
    
    
    
    cluster.fit(X)
    #predicted = cluster.predict(X=X, n_ahead=n_ahead)
    predicted = transformer.inverse_transform( cluster.predict(X=X, n_ahead=n_ahead))
    X = X.transpose()
    df_prediction = pd.DataFrame(predicted).transpose()
    df_prediction.columns  = true_values.columns 
    df_prediction.index = true_values.index
    
    # Transform prediction for output
    #MODEL
    df_prediction.insert(0,'RMSE',math.sqrt(mean_squared_error(true_values,df_prediction)))
    
    df_prediction.insert(0,'model','trmf')
    #METHOD
    df_prediction.insert(0,'method','clustered')
    #STEP
    df_prediction.insert(0,'step','before_classifier')
    # CLUSTER
    df_prediction.insert(0,'cluster_id','cluster_'+str(cluster_number)+'_F')
    #Wide to logn
    df_prediction = df_prediction.reset_index().melt(id_vars =['id','cluster_id','step','method','model','RMSE']).rename(index = str, columns={'variable':'date','value':'predicted_value'})
    true_values=true_values.reset_index().melt(id_vars=['id']).rename(index = str, columns={'variable':'date','value':'true_value'})
    
    #break
    result = result.append(df_prediction.merge(true_values,left_on = ['id','date'],right_on=['id','date']))

CPU times: user 1min 14s, sys: 7min 45s, total: 8min 59s
Wall time: 1min 46s


## 3.2 Theta

### 3.2.1 Recency

In [33]:
%%time
base = importr('forecast')
# Create Theta Models
for cluster_number in df_with_predicted_labels.clusters_r.unique():
    X = train_targets.loc[train_targets.clusters_r == cluster_number].drop(columns = ['clusters_m','clusters_f'])
    true_values = test_targets.loc[ X.index]
    # Scaling
    scl =MinMaxScaler() #StandardScaler(with_mean=True, with_std=True)
    X = scl.fit_transform(X)
    #THETA
    df_prediction = pd.DataFrame(columns = test_targets.columns)

    train_targets_scaled = pd.DataFrame(X)
    train_targets_scaled.index = true_values.index


    for index, row in train_targets_scaled.iterrows():
        res = robjects.FloatVector(row.drop(columns = ['uid']))
        r_prediction = base.thetaf(res,70)[1]
        df_prediction.loc[index] = r_prediction
    #Transform prediction for output 
    #MODEL
    df_prediction.insert(0,'RMSE',math.sqrt(mean_squared_error(true_values,df_prediction)))
    
    df_prediction.insert(0,'model','theta')
    #METHOD
    df_prediction.insert(0,'method','clustered')
    #STEP
    df_prediction.insert(0,'step','after_classifier')
    # CLUSTER
    df_prediction.insert(0,'cluster_id','cluster_'+str(cluster_number)+'_R')
    #Wide to logn
    df_prediction = df_prediction.reset_index().melt(id_vars =['index','cluster_id','step','method','model','RMSE']).rename(index = str, columns={'variable':'date','value':'predicted_value','index':'id'})
    true_values=true_values.reset_index().melt(id_vars=['id']).rename(index = str, columns={'variable':'date','value':'true_value'})
    
    #break
    result = result.append(df_prediction.merge(true_values,left_on = ['id','date'],right_on=['id','date']))

CPU times: user 52.3 s, sys: 1.69 s, total: 53.9 s
Wall time: 53.9 s


### 3.2.2 Monetary

In [34]:
%%time
# Create Theta Models
for cluster_number in df_with_predicted_labels.clusters_m.unique():
    X = train_targets.loc[train_targets.clusters_m == cluster_number].drop(columns = ['clusters_r','clusters_f'])
    true_values = test_targets.loc[ X.index]
    # Scaling
    scl =MinMaxScaler() #StandardScaler(with_mean=True, with_std=True)
    X = scl.fit_transform(X)
    #THETA
    df_prediction = pd.DataFrame(columns = test_targets.columns)

    train_targets_scaled = pd.DataFrame(X)
    train_targets_scaled.index = true_values.index


    for index, row in train_targets_scaled.iterrows():
        res = robjects.FloatVector(row.drop(columns = ['uid']))
        r_prediction = base.thetaf(res,70)[1]
        df_prediction.loc[index] = r_prediction
    #Transform prediction for output 
    #MODEL
    df_prediction.insert(0,'RMSE',math.sqrt(mean_squared_error(true_values,df_prediction)))
    
    df_prediction.insert(0,'model','theta')
    #METHOD
    df_prediction.insert(0,'method','clustered')
    #STEP
    df_prediction.insert(0,'step','after_classifier')
    # CLUSTER
    df_prediction.insert(0,'cluster_id','cluster_'+str(cluster_number)+'_M')
    #Wide to logn
    df_prediction = df_prediction.reset_index().melt(id_vars =['index','cluster_id','step','method','model','RMSE']).rename(index = str, columns={'variable':'date','value':'predicted_value','index':'id'})
    true_values=true_values.reset_index().melt(id_vars=['id']).rename(index = str, columns={'variable':'date','value':'true_value'})
    
    #break
    result = result.append(df_prediction.merge(true_values,left_on = ['id','date'],right_on=['id','date']))

CPU times: user 50.1 s, sys: 228 ms, total: 50.3 s
Wall time: 50.3 s


### 3.2.3 Frequency

In [35]:
%%time
# Create Theta Models
for cluster_number in df_with_predicted_labels.clusters_f.unique():
    X = train_targets.loc[train_targets.clusters_f == cluster_number].drop(columns = ['clusters_r','clusters_m'])
    true_values = test_targets.loc[ X.index]
    # Scaling
    scl =MinMaxScaler() #StandardScaler(with_mean=True, with_std=True)
    X = scl.fit_transform(X)
    #THETA
    df_prediction = pd.DataFrame(columns = test_targets.columns)

    train_targets_scaled = pd.DataFrame(X)
    train_targets_scaled.index = true_values.index


    for index, row in train_targets_scaled.iterrows():
        res = robjects.FloatVector(row.drop(columns = ['uid']))
        r_prediction = base.thetaf(res,70)[1]
        df_prediction.loc[index] = r_prediction
    #Transform prediction for output 
    #MODEL
    df_prediction.insert(0,'RMSE',math.sqrt(mean_squared_error(true_values,df_prediction)))
    
    df_prediction.insert(0,'model','theta')
    #METHOD
    df_prediction.insert(0,'method','clustered')
    #STEP
    df_prediction.insert(0,'step','after_classifier')
    # CLUSTER
    df_prediction.insert(0,'cluster_id','cluster_'+str(cluster_number)+'_F')
    #Wide to logn
    df_prediction = df_prediction.reset_index().melt(id_vars =['index','cluster_id','step','method','model','RMSE']).rename(index = str, columns={'variable':'date','value':'predicted_value','index':'id'})
    true_values=true_values.reset_index().melt(id_vars=['id']).rename(index = str, columns={'variable':'date','value':'true_value'})
    result = result.append(df_prediction.merge(true_values,left_on = ['id','date'],right_on=['id','date']))

CPU times: user 51.6 s, sys: 1.66 s, total: 53.3 s
Wall time: 53.3 s


### Save to file 

In [36]:
result.shape

(2940000, 9)

In [37]:
result.sample(5)

,id,cluster_id,step,method,model,RMSE,date,predicted_value,true_value
257325,15571,cluster_0_F,after_classifier,clustered,theta,2.091287,1998-06-22,-0.000110,0.0
122073,9238,cluster_0_M,before_classifier,clustered,trmf,2.907609,1998-05-16,0.296918,0.0
25946,6943,cluster_0_M,before_classifier,clustered,trmf,2.907609,1998-04-27,0.030126,0.0
16414,17050,cluster_0_F,before_classifier,clustered,trmf,2.089774,1998-04-25,0.053962,0.0
260061,18647,cluster_0_F,after_classifier,clustered,theta,2.091287,1998-06-23,-0.000512,0.0


In [38]:
result.to_csv(prediction_path)

In [39]:
# To get full analysis, please run "combine_predictions.ipynb" file, which combines the prediction files of all approaches